# OIDC Device Flow Authentication for JupyterLite

This notebook demonstrates how to authenticate with the Mat3ra API using OIDC Device Code Flow.

## 📖 First Time Using This Notebook?

**Quick visual guide**: [VISUAL_GUIDE.md](./VISUAL_GUIDE.md) - See exactly what each screen looks like!

**Detailed walkthrough**: [AUTHENTICATION_FLOW.md](./AUTHENTICATION_FLOW.md)

These guides explain:
- ✅ What happens when you click the authorization button
- ✅ What the login page looks like (with ASCII art examples!)
- ✅ Step-by-step expectations for each screen
- ✅ Common issues and solutions
- ✅ Timeline: How long each step takes

**TL;DR**: Click the green button → Login → Enter code → Approve → Done!

## Prerequisites

Before running this notebook, ensure:

1. **✅ OIDC Server is Running**: 
   ```bash
   ./run-meteor.sh -p=3000 -d=local
   ```
   Wait for: `App running at: http://localhost:3000/`

2. **✅ You have a user account**: 
   - Either already created at http://localhost:3000
   - Or you'll create one during the authentication flow

3. **✅ Default credentials are configured** (already set up in your app):
   - Client ID: `default-client`
   - Client Secret: `default-secret`
   - These are pre-configured in `src/application/settings.json`

### What Will Happen

When you run authentication:
1. **A green button appears** → Click it to open the login page
2. **Login page opens** → Login with your credentials (or create account)
3. **Enter device code** → Copy/paste the code shown in the notebook
4. **Approve authorization** → Grant access to your account
5. **Token received** → Notebook automatically gets your access token

## Device Code Flow Overview

```
┌─────────┐                              ┌──────────┐
│ Jupyter │                              │   OIDC   │
│ Notebook│                              │  Server  │
└────┬────┘                              └─────┬────┘
     │                                         │
     │ 1. Request device code                 │
     │────────────────────────────────────────>│
     │                                         │
     │ 2. Return device_code & user_code      │
     │<────────────────────────────────────────│
     │                                         │
     │ 3. Display URL & code to user          │
     │                                         │
┌────┴────┐                              ┌─────┴────┐
│  User   │                              │   OIDC   │
│ Browser │                              │  Server  │
└────┬────┘                              └─────┬────┘
     │                                         │
     │ 4. Navigate to URL, enter code         │
     │────────────────────────────────────────>│
     │                                         │
     │ 5. Login & authorize                   │
     │<───────────────────────────────────────>│
     │                                         │
┌────┴────┐                              ┌─────┴────┐
│ Jupyter │                              │   OIDC   │
│ Notebook│                              │  Server  │
└────┬────┘                              └─────┬────┘
     │                                         │
     │ 6. Poll for token                      │
     │────────────────────────────────────────>│
     │                                         │
     │ 7. Return access_token                 │
     │<────────────────────────────────────────│
     │                                         │
```

## Configuration

Update these settings to match your environment:

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("requests")
    await micropip.install("ipywidgets")
    print("✓ Dependencies installed")

In [ ]:
# OIDC Configuration
OIDC_BASE_URL = "http://localhost:3000/oidc"  # Your OIDC server URL
CLIENT_ID = "default-client"                  # Your OAuth client ID
CLIENT_SECRET = "default-secret"              # Your OAuth client secret
SCOPE = "openid profile email"                # Requested scopes

print("✓ Configuration loaded")
print(f"  OIDC URL: {OIDC_BASE_URL}")
print(f"  Client ID: {CLIENT_ID}")

## Install Dependencies

Install required packages for HTTP requests:

## Pre-Flight Check

Let's verify the OIDC server is running before we attempt authentication:

In [ ]:
import requests

def check_server_status():
    """Check if the OIDC server is running and accessible."""
    base_url = OIDC_BASE_URL.replace('/oidc', '')
    
    print(f"🔍 Checking server status...")
    print(f"   Server URL: {base_url}")
    
    try:
        # Check if server is responding
        response = requests.get(f"{base_url}/healthcheck", timeout=5)
        
        if response.status_code == 200:
            print(f"✅ Server is running and responding!")
            print(f"   Status: OK")
            
            # Check OIDC discovery endpoint
            try:
                oidc_response = requests.get(
                    f"{OIDC_BASE_URL}/.well-known/openid-configuration",
                    timeout=5
                )
                if oidc_response.status_code == 200:
                    config = oidc_response.json()
                    print(f"✅ OIDC server is configured!")
                    print(f"   Issuer: {config.get('issuer', 'N/A')}")
                    print(f"   Device endpoint: {config.get('device_authorization_endpoint', 'N/A')}")
                    return True
                else:
                    print(f"⚠️  OIDC endpoints not accessible")
                    return False
            except Exception as e:
                print(f"⚠️  OIDC discovery failed: {e}")
                return False
        else:
            print(f"⚠️  Server responded with status: {response.status_code}")
            return False
            
    except requests.exceptions.ConnectionError:
        print(f"❌ Cannot connect to server at {base_url}")
        print(f"\n💡 To start the server, run:")
        print(f"   cd /Users/mat3ra/code/GREEN/stack/web-app")
        print(f"   ./run-meteor.sh -p=3000 -d=local")
        print(f"\n   Wait for: 'App running at: http://localhost:3000/'")
        return False
    except requests.exceptions.Timeout:
        print(f"❌ Server connection timed out")
        print(f"   The server might be starting up. Wait a moment and try again.")
        return False
    except Exception as e:
        print(f"❌ Error checking server: {e}")
        return False

# Run the check
server_ok = check_server_status()

if server_ok:
    print(f"\n✨ Ready to authenticate!")
else:
    print(f"\n⏸️  Please start the server first, then run this cell again.")

## Authentication Function

This function implements the complete device code flow:

In [ ]:
import asyncio
import os
import time
import requests
from IPython.display import Javascript, display


async def authenticate_device_flow(
    oidc_base_url=OIDC_BASE_URL,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE,
):
    device_response = requests.post(
        f"{oidc_base_url}/device/auth",
        data={"client_id": client_id, "client_secret": client_secret, "scope": scope},
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        timeout=10,
    )
    device_response.raise_for_status()

    device_data = device_response.json()
    device_code = device_data["device_code"]
    user_code = device_data["user_code"]
    verification_uri_complete = device_data.get("verification_uri_complete", device_data["verification_uri"])
    polling_interval_seconds = int(device_data.get("interval", 5))
    expires_in_seconds = int(device_data.get("expires_in", 600))

    # JupyterLite: window.open must happen during cell execution to avoid popup blocker.
    display(Javascript(
        "alert("
        + repr(f"Open the login page and enter this code:\n\n{user_code}")
        + ");"
        + f"window.open({verification_uri_complete!r}, '_blank');"
    ))

    start_time_seconds = time.time()
    while time.time() - start_time_seconds < expires_in_seconds:
        token_response = requests.post(
            f"{oidc_base_url}/token",
            data={
                "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
                "device_code": device_code,
                "client_id": client_id,
                "client_secret": client_secret,
            },
            headers={"Content-Type": "application/x-www-form-urlencoded"},
            timeout=10,
        )

        if token_response.status_code == 200:
            token_data = token_response.json()
            os.environ["OIDC_ACCESS_TOKEN"] = token_data["access_token"]
            if "refresh_token" in token_data:
                os.environ["OIDC_REFRESH_TOKEN"] = token_data["refresh_token"]
            return token_data

        error_data = token_response.json() if token_response.headers.get("content-type", "").startswith("application/json") else {}
        error_code = error_data.get("error", "")

        if error_code == "slow_down":
            polling_interval_seconds += 5
        elif error_code != "authorization_pending":
            raise Exception(error_data.get("error_description") or error_code or token_response.text)

        await asyncio.sleep(polling_interval_seconds)

    raise Exception("Timeout waiting for authorization.")


## Run Authentication

**Important**: Before running this cell:
1. Make sure the server status check above passed ✅
2. Keep your browser ready - a login page will open automatically
3. Have your account credentials ready

When you run this cell:
- A colorful authorization box will appear with a **green button**
- **Click the green button** to open the login/authorization page
- **Login** if you're not already logged in
- **Enter the code** shown in the box
- **Approve** the authorization
- Come back here and wait - the token will appear automatically!

Execute the device flow authentication:

In [ ]:
try:
    token_data = await authenticate_device_flow()
    
    print("\n📋 Token Information:")
    print(f"  Token Type: {token_data.get('token_type', 'N/A')}")
    print(f"  Expires In: {token_data.get('expires_in', 'N/A')} seconds")
    print(f"  Access Token (first 50 chars): {token_data['access_token'][:50]}...")
    if 'refresh_token' in token_data:
        print(f"  Refresh Token: Available")
    
except Exception as e:
    print(f"\n❌ Authentication failed: {e}")

## Using the Access Token

Now you can use the access token to make authenticated API requests:

In [ ]:
def call_api(endpoint, method="GET", data=None):
    """
    Make an authenticated API request.
    
    Args:
        endpoint: API endpoint (e.g., '/api/v1/users/me')
        method: HTTP method (GET, POST, PUT, DELETE)
        data: Request body for POST/PUT requests
    
    Returns:
        Response data
    """
    access_token = os.environ.get('OIDC_ACCESS_TOKEN')
    
    if not access_token:
        raise Exception("No access token found. Please authenticate first.")
    
    # Construct full URL
    base_url = OIDC_BASE_URL.replace('/oidc', '')  # Remove /oidc suffix
    url = f"{base_url}{endpoint}"
    
    # Make request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        json=data,
        timeout=30,
    )
    
    if response.status_code >= 400:
        raise Exception(f"API request failed ({response.status_code}): {response.text}")
    
    return response.json() if response.headers.get('content-type', '').startswith('application/json') else response.text


print("✓ API helper function defined")

### Example: Get Current User Info

In [ ]:
try:
    user_info = call_api('/api/v1/users/me')
    print("👤 User Information:")
    print(user_info)
except Exception as e:
    print(f"❌ Error: {e}")

### Example: List Projects

In [ ]:
try:
    projects = call_api('/api/v1/projects')
    print(f"📁 Projects ({len(projects)} total):")
    for project in projects[:5]:  # Show first 5
        print(f"  - {project.get('name', 'Unnamed')} (ID: {project.get('_id', 'N/A')})")
except Exception as e:
    print(f"❌ Error: {e}")

## Token Management

Check token status and refresh if needed:

In [ ]:
def check_token_status():
    """Check if we have a valid access token."""
    access_token = os.environ.get('OIDC_ACCESS_TOKEN')
    refresh_token = os.environ.get('OIDC_REFRESH_TOKEN')
    
    print("🔑 Token Status:")
    print(f"  Access Token: {'✓ Present' if access_token else '✗ Missing'}")
    print(f"  Refresh Token: {'✓ Present' if refresh_token else '✗ Missing'}")
    
    if access_token:
        print(f"  Access Token (first 30 chars): {access_token[:30]}...")
    
    return bool(access_token)


def clear_tokens():
    """Clear stored tokens."""
    os.environ.pop('OIDC_ACCESS_TOKEN', None)
    os.environ.pop('OIDC_REFRESH_TOKEN', None)
    print("✓ Tokens cleared")


check_token_status()